## 모듈 임포트

In [1]:
!pip install konlpy
from sklearn.datasets import load_files
from bs4 import BeautifulSoup
import re
import konlpy
import pandas as pd
import nltk
import tensorflow as tf
tf.random.set_seed(777) #하이퍼파라미터 튜닝을 위해 실행시 마다 변수가 같은 초기값 가지게 하기
from sklearn.model_selection import train_test_split
import numpy as np

okt = konlpy.tag.Okt()

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 45.0 MB/s 
     |████████████████████████████████| 86 kB 2.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


## 모델 다양화 위해 자연어 프로세싱 한 데이터 셋도 생성

In [3]:
def clean_korean_documents(documents):
    #텍스트 정제 (HTML 태그 제거)
    for i, document in enumerate(documents):
        document = BeautifulSoup(document, 'html.parser').text 
        documents[i] = document

    #텍스트 정제 (특수기호 제거)
    # for i, document in enumerate(documents):
    #     document = re.sub(r'[^ ㄱ-ㅣ가-힣]', '', document) #특수기호 제거, 정규 표현식
    #     documents[i] = document

    #텍스트 정제 (형태소 분석)
    for i, document in enumerate(documents):
        okt = konlpy.tag.Okt()
        clean_words = []
        for word in okt.pos(document, stem=True): #어간 추출
            if word[1] in ['Noun', 'Verb', 'Adjective', 'Alpha', 'Number']: #명사, 동사, 형용사
                clean_words.append(word[0])
        document = ' '.join(clean_words)
        documents[i] = document

    # #텍스트 정제 (불용어 제거)
    df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
    df[0] = df[0].apply(lambda x: x.strip())
    stopwords = df[0].to_numpy()
    nltk.download('punkt')
    for i, document in enumerate(documents):
        clean_words = [] 
        for word in nltk.tokenize.word_tokenize(document): 
            if word not in stopwords: #불용어 제거
                clean_words.append(word)
        documents[i] = ' '.join(clean_words)  

    return documents


##########데이터 로드

naver_news = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/train_data.csv').drop(columns = ['index'])
naver_news_test = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data.csv').drop(columns = ['index'])
labels = ['정치', '경제', '사회', '생활/문화', '세계', '기술/IT', '연예', '스포츠']

##########데이터 분석

##########데이터 전처리

x_data = naver_news.title
x_test_data = naver_news_test.title
y_data = naver_news.topic_idx
#print(len(x_data)) #1600
#x_data = x_data[:800] #데이터를 800개로 제한
#y_data = y_data[:800]

#vocab_size = len(tokenizer.word_index) + 1 #어휘수
vocab_size = 2000 + 1 #어휘수
x_data = clean_korean_documents(x_data) #텍스트 정제
x_test_data = clean_korean_documents(x_test_data)

naver_news = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/train_data.csv').drop(columns = ['index'])
naver_news['processed_title'] = x_data
naver_news
naver_news.to_csv('/content/drive/MyDrive/데이콘 뉴스분석/train_data_processed.csv')

naver_news_test = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data.csv').drop(columns = ['index'])
naver_news_test['processed_title'] = x_test_data
naver_news_test
naver_news_test.to_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data_processed.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 모델 생성

## dnn 모델

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential
import numpy as np
import torch
import random

random.seed(1)
torch.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(1)
tf.random.set_seed(1)
np.random.seed(1)


#processed dataset이 점수 더 잘나와서 processed dataset으로 모델 생성
#실제 앙상블에는 processed 데이터, 기존 데이터로 만든 모델 둘다 사용
train      = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/train_data_processed.csv')
test       = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/test_data_processed.csv')
submission = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/sample_submission.csv')
topic_dict = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/topic_dict.csv')

In [2]:
train_text = train["processed_title"].tolist()
test_text = test["processed_title"].tolist()
train_label = np.asarray(train.topic_idx)

tfidf = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1, 2), max_features=150000, binary=False)

tfidf.fit(train_text)

train_tf_text = tfidf.transform(train_text).astype('float32')
test_tf_text  = tfidf.transform(test_text).astype('float32')

In [3]:
def dnn_model():
  model = Sequential()
  model.add(Dense(128, input_dim = 150000, activation = "relu"))
  model.add(Dropout(0.8))
  model.add(Dense(7, activation = "softmax"))
  return model

In [4]:
dnn_model = dnn_model()
dnn_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.optimizers.Adam(0.001), metrics = ['accuracy'])

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=3)

In [6]:
history = dnn_model.fit(x = train_tf_text[:40000], y = train_label[:40000],
                    validation_data =(train_tf_text[40000:], train_label[40000:]),
                    epochs = 20,callbacks = [earlystop_callback])

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


1250/1250 [==============================] - 18s 13ms/step - loss: 1.2220 - accuracy: 0.6283 - val_loss: 0.8235 - val_accuracy: 0.8035
Epoch 2/20
1250/1250 [==============================] - 16s 12ms/step - loss: 0.5498 - accuracy: 0.8397 - val_loss: 0.5669 - val_accuracy: 0.8288
Epoch 3/20
1250/1250 [==============================] - 16s 13ms/step - loss: 0.3701 - accuracy: 0.8902 - val_loss: 0.5112 - val_accuracy: 0.8345
Epoch 4/20
1250/1250 [==============================] - 16s 13ms/step - loss: 0.2742 - accuracy: 0.9187 - val_loss: 0.4926 - val_accuracy: 0.8329
Epoch 5/20
1250/1250 [==============================] - 16s 12ms/step - loss: 0.2165 - accuracy: 0.9377 - val_loss: 0.4951 - val_accuracy: 0.8297
Epoch 6/20
1250/1250 [==============================] - 16s 12ms/step - loss: 0.1704 - accuracy: 0.9503 - val_loss: 0.5017 - val_accuracy: 0.8309


In [7]:
dnn_pred = dnn_model.predict(test_tf_text)
pred = np.argmax(dnn_pred, axis = 1)

np.save('/content/drive/MyDrive/데이콘 뉴스분석/dnn_pred.npy', dnn_pred)

## PLM 모델

- ### 대회에서 금지된 KLUE 데이터로 학습된 모델들을 제외한 다양한 모델 사용
- ### 노트북 간소화를 위해 모든 모델 코드를 다 업로드 하진 않았음
- ### kykim, funnel, kobert, skt/kogpt2-base-v2, electra, snunlp, seongju 모델 사용
- ### 데이터를 processed데이터, 기존 데이터로 나눠서 학습하고 batch 사이즈나 epoch수 조절, 학습 데이터 셋 비율 조정 등을 통해 다양한 모델 예측 결과를 npy로 저장

In [9]:
!pip install transformers
import pandas as pd
import numpy as np
import warnings

import tensorflow as tf

import torch
import torch.nn as nn

import numpy as np
import torch
import random

# Install transformers library.
!pip install -q git+https://github.com/huggingface/transformers.git
# Install helper functions.
!pip install -q git+https://github.com/gmihaila/ml_things.git
import io
import os
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (AutoConfig, 
                          AutoModelForSequenceClassification, 
                          AutoTokenizer, AdamW, 
                          get_linear_schedule_with_warmup,
                          set_seed,
                          )
random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)
tf.random.set_seed(0)
np.random.seed(0)
warnings.filterwarnings('ignore')

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 64 kB 2.5 MB/s 
     |████████████████████████████████| 10.3 MB 19.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [10]:

from transformers import AutoTokenizer, AutoModelForCausalLM
  
from transformers import GPT2Tokenizer, GPTNeoForSequenceClassification


# tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-1.3B')
# model3 = GPTNeoForSequenceClassification.from_pretrained('EleutherAI/gpt-neo-1.3B')
# #model3.score = torch.nn.Linear(1024, 7)
# #model3.classifier.out_proj = torch.nn.Linear(1024, 7)

from transformers import AutoTokenizer, AutoModel
from transformers import FunnelTokenizer, FunnelForSequenceClassification  
from transformers import BertTokenizerFast, BertModel,AutoModelForSequenceClassification

#from transformers import AutoTokenizer, BertForMultiLabelClassification
  
#tokenizer = AutoTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")

#model3 = AutoModelForSequenceClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")

tokenizer = AutoTokenizer.from_pretrained("kykim/funnel-kor-base")
model3 = AutoModelForSequenceClassification.from_pretrained("kykim/funnel-kor-base")
model3.classifier.linear_out = torch.nn.Linear(768, 7)
model3.cuda()

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/funnel-kor-base were not used when initializing FunnelForSequenceClassification: ['decoder.layers.1.ffn.linear_2.bias', 'decoder.layers.1.attention.seg_embed', 'decoder.layers.0.attention.post_proj.weight', 'decoder.layers.1.ffn.layer_norm.weight', 'decoder.layers.0.attention.v_head.weight', 'decoder.layers.1.attention.layer_norm.bias', 'decoder.layers.1.attention.v_head.weight', 'decoder.layers.1.ffn.layer_norm.bias', 'decoder.layers.1.attention.q_head.weight', 'decoder.layers.0.ffn.linear_2.bias', 'decoder.layers.1.attention.k_head.bias', 'decoder.layers.1.attention.post_proj.bias', 'decoder.layers.0.attention.r_kernel', 'decoder.layers.1.attention.k_head.weight', 'decoder.layers.1.attention.r_w_bias', 'decoder.layers.1.attention.r_s_bias', 'decoder.layers.0.attention.k_head.bias', 'decoder.layers.1.attention.v_head.bias', 'decoder.layers.0.ffn.linear_1.bias', 'decoder.layers.0.ffn.layer_norm.weight', 'decoder.layers.0.ffn.linear_2.weight

FunnelForSequenceClassification(
  (funnel): FunnelBaseModel(
    (embeddings): FunnelEmbeddings(
      (word_embeddings): Embedding(42000, 768)
      (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): FunnelEncoder(
      (attention_structure): FunnelAttentionStructure(
        (sin_dropout): Dropout(p=0.1, inplace=False)
        (cos_dropout): Dropout(p=0.1, inplace=False)
      )
      (blocks): ModuleList(
        (0): ModuleList(
          (0): FunnelLayer(
            (attention): FunnelRelMultiheadAttention(
              (hidden_dropout): Dropout(p=0.1, inplace=False)
              (attention_dropout): Dropout(p=0.1, inplace=False)
              (q_head): Linear(in_features=768, out_features=768, bias=False)
              (k_head): Linear(in_features=768, out_features=768, bias=True)
              (v_head): Linear(in_features=768, out_features=768, bias=True)
              (post_proj): Linear(i

In [11]:
class TrainDataset(Dataset):
    def __init__(self, data, tokenizer, max_seq_len=44):
        self.data = data
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        record = self.data.iloc[index]
        document, label = str(record['title']), int(record['topic_idx'])
        tokens = self.tokenizer.tokenize(document)
        encoder_input_id = self.tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(encoder_input_id)
        if len(encoder_input_id) < self.max_seq_len:
            while len(encoder_input_id) < self.max_seq_len:
                encoder_input_id += [tokenizer.convert_tokens_to_ids('<pad>')]
                attention_mask += [0]
        else:
            encoder_input_id = encoder_input_id[:self.max_seq_len - 1] + [
                self.tokenizer.eos_token_id]
            attention_mask = attention_mask[:self.max_seq_len]
        return {'input_ids': np.array(encoder_input_id, dtype=np.int_),
                'attention_mask': np.array(attention_mask, dtype=np.float),
                'labels': np.array(label, dtype=np.int_)}
    
class TestDataset(Dataset):
    def __init__(self, data, tokenizer, max_seq_len=44):
        self.data = data
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        record = self.data.iloc[index]
        document = str(record['title'])
        tokens = self.tokenizer.tokenize(document)
        encoder_input_id = self.tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(encoder_input_id)
        if len(encoder_input_id) < self.max_seq_len:
            while len(encoder_input_id) < self.max_seq_len:
                encoder_input_id += [tokenizer.convert_tokens_to_ids('<pad>')]
                attention_mask += [0]
        else:
            encoder_input_id = encoder_input_id[:self.max_seq_len - 1] + [
                self.tokenizer.eos_token_id]
            attention_mask = attention_mask[:self.max_seq_len]
        return {'input_ids': np.array(encoder_input_id, dtype=np.int_),
                'attention_mask': np.array(attention_mask, dtype=np.float)}

epochs = 5
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')


tr = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/train_data_processed.csv', index_col='Unnamed: 0')

# train loader
train_ds = TrainDataset(tr[4500:], tokenizer)
train_dataloader = DataLoader(train_ds, batch_size=batch_size, num_workers=0, shuffle=False)
valid_ds = TrainDataset(tr[:4500],tokenizer)
valid_dataloader = DataLoader(valid_ds, batch_size=batch_size, num_workers=0, shuffle=False)

def train(dataloader, optimizer_, scheduler_, device_):
  r"""
  Train pytorch model on a single pass through the data loader.

  It will use the global variable `model` which is the transformer model 
  loaded on `_device` that we want to train on.

  This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

  Arguments:

      dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

      optimizer_ (:obj:`transformers.optimization.AdamW`):
          Optimizer used for training.

      scheduler_ (:obj:`torch.optim.lr_scheduler.LambdaLR`):
          PyTorch scheduler.

      device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.

  Returns:

      :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss].
  """

  # Use global variable for model.
  global model3

  # Tracking variables.
  predictions_labels = []
  true_labels = []
  # Total loss for this epoch.
  total_loss = 0

  # Put the model3 into training mode.
  model3.train()

  # For each batch of training data...
  for batch in tqdm(dataloader, total=len(dataloader)):

    # Add original labels - use later for evaluation.
    true_labels += batch['labels'].numpy().flatten().tolist()
    
    # move batch to device
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}
    
    # Always clear any previously calculated gradients before performing a
    # backward pass.
    model3.zero_grad()

    # Perform a forward pass (evaluate the model3 on this training batch).
    # This will return the loss (rather than the model3 output) because we
    # have provided the `labels`.
    # The documentation for this a bert model3 function is here: 
    # https://huggingface.co/transformers/v2.2.0/model3_doc/bert.html#transformers.BertForSequenceClassification
    outputs = model3(**batch)

    # The call to `model3` always returns a tuple, so we need to pull the 
    # loss value out of the tuple along with the logits. We will use logits
    # later to calculate training accuracy.
    loss, logits = outputs[:2]

    # Accumulate the training loss over all of the batches so that we can
    # calculate the average loss at the end. `loss` is a Tensor containing a
    # single value; the `.item()` function just returns the Python value 
    # from the tensor.
    total_loss += loss.item()

    # Perform a backward pass to calculate the gradients.
    loss.backward()

    # Clip the norm of the gradients to 1.0.
    # This is to help prevent the "exploding gradients" problem.
    torch.nn.utils.clip_grad_norm_(model3.parameters(), 1.0)

    # Update parameters and take a step using the computed gradient.
    # The optimizer dictates the "update rule"--how the parameters are
    # modified based on their gradients, the learning rate, etc.
    optimizer.step()

    # Update the learning rate.
    scheduler.step()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    # Convert these logits to list of predicted labels values.
    predictions_labels += logits.argmax(axis=-1).flatten().tolist()

  # Calculate the average loss over the training data.
  avg_epoch_loss = total_loss / len(dataloader)
  
  # Return all true labels and prediction for future evaluations.
  return true_labels, predictions_labels, avg_epoch_loss



def validation(dataloader, device_):
  r"""Validation function to evaluate model3 performance on a 
  separate set of data.

  This function will return the true and predicted labels so we can use later
  to evaluate the model3's performance.

  This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model3 - `model3(**batch)`.

  Arguments:

    dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

    device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model3.

  Returns:
    
    :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss]
  """

  # Use global variable for model3.
  global model3

  # Tracking variables
  predictions_labels = []
  true_labels = []
  #total loss for this epoch.
  total_loss = 0

  # Put the model3 in evaluation mode--the dropout layers behave differently
  # during evaluation.
  model3.eval()

  # Evaluate data for one epoch
  for batch in tqdm(dataloader, total=len(dataloader)):

    # add original labels
    true_labels += batch['labels'].numpy().flatten().tolist()

    # move batch to device
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

    # Telling the model3 not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model3` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model3_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model3(**batch)

        # The call to `model3` always returns a tuple, so we need to pull the 
        # loss value out of the tuple along with the logits. We will use logits
        # later to to calculate training accuracy.
        loss, logits = outputs[:2]
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()
        
        # get predicitons to list
        predict_content = logits.argmax(axis=-1).flatten().tolist()

        # update list
        predictions_labels += predict_content

  # Calculate the average loss over the training data.
  avg_epoch_loss = total_loss / len(dataloader)

  # Return all true labels and prediciton for future evaluations.
  return true_labels, predictions_labels, avg_epoch_loss
# optimizer
# train loader
train_ds = TrainDataset(tr, tokenizer)
loader = DataLoader(train_ds, batch_size=batch_size, num_workers=0, shuffle=True)
optimizer = torch.optim.AdamW(model3.parameters(), lr=3e-05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 5, )
loss_fn = torch.nn.CrossEntropyLoss()

In [12]:
model3.train()
for e in range(epochs):
    total_loss = 0
    for batch in loader:
        optimizer.zero_grad()
        ids, atts, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        ids = torch.tensor(ids).long().cuda()
        atts = torch.tensor(atts).long().cuda()
        labels = torch.tensor(labels).long().cuda()
        pred = model3(ids, attention_mask=atts)
        loss = loss_fn(pred[0], labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model3.parameters(), 1.)
        optimizer.step()
        total_loss += loss.item()
        
    scheduler.step()
    print(e, total_loss)

0 1965.5971123576164
1 1076.8313606381416
2 870.6604427173734
3 714.742069400847
4 609.6467189751565


In [13]:
te = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data_processed.csv', index_col='Unnamed: 0')

test_ds = TestDataset(te, tokenizer)
test_loader = DataLoader(test_ds, 8)

In [14]:
preds = []
model3_pred = np.empty((0,7))
model3.eval()

for b in tqdm(test_loader):
    ids, atts = b['input_ids'], b['attention_mask']
    ids = torch.tensor(ids).long().cuda()
    atts = torch.tensor(atts).long().cuda()
    pred = model3(ids, attention_mask=atts)
    model3_pred = np.append(model3_pred, pred[0].detach().cpu().numpy(), axis = 0)
    preds += list(np.argmax(pred[0].detach().cpu().numpy(), 1))

  0%|          | 0/1142 [00:00<?, ?it/s]

In [15]:
np.save('/content/drive/MyDrive/데이콘 뉴스분석/kykim_5.npy', model3_pred)

## 머신러닝 모델

- ### 머신러닝 모델 중 확률론적 모델은 확률을, 판별함수 모형들은 결과 항목의 확률은 1, 나머지 확률은 0인 array로 만들어 앙상블에 활용
- ### 노트북 공간상 최종 앙상블에 사용되는 모델들만 작성

In [17]:
from sklearn.linear_model import SGDClassifier#3000
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
train_df = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/train_data_processed.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data_processed.csv', index_col='Unnamed: 0')
vect = TfidfVectorizer(max_features =5000)
X = vect.fit_transform(train_df['processed_title'])
x = vect.transform(test['processed_title'])
model = SGDClassifier(loss = 'log')
model.fit(X,train_df['topic_idx'])


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [18]:
from sklearn.linear_model import SGDClassifier#3000
train_df = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/train_data_processed.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data_processed.csv', index_col='Unnamed: 0')
vect = TfidfVectorizer(max_features =5000)
from lightgbm import LGBMClassifier
#model3 = LGBMClassifier(random_state = 42,learning_rate=0.1)
model3 = LGBMClassifier(learning_rate=0.1)
model3.fit(X,train_df['topic_idx'])



LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [19]:
from sklearn.linear_model import SGDClassifier#3000
train_df = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/train_data_processed.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data_processed.csv', index_col='Unnamed: 0')
vect = TfidfVectorizer(max_features =5000)
X = vect.fit_transform(train_df['processed_title'])
x = vect.transform(test['processed_title'])
model5 = SGDClassifier(loss = 'modified_huber',penalty = 'l2')
model5.fit(X,train_df['topic_idx'])




SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [33]:
from sklearn.linear_model import SGDClassifier#3000
train_df = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/train_data_processed.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data_processed.csv', index_col='Unnamed: 0')
vect = TfidfVectorizer()
X2 = vect.fit_transform(train_df['processed_title'])
x2 = vect.transform(test['processed_title'])
model5_2 = SGDClassifier(loss = 'modified_huber',penalty = 'l2')
model5_2.fit(X,train_df['topic_idx'])


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [39]:
from sklearn.naive_bayes import MultinomialNB
vect = TfidfVectorizer(max_features =5000)
X = vect.fit_transform(train_df['processed_title'])
x = vect.transform(test['processed_title'])
model8 = MultinomialNB()
model8.fit(X,train_df['topic_idx'])


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
from sklearn.ensemble import RandomForestClassifier
vect = TfidfVectorizer(max_features =5000)
X = vect.fit_transform(train_df['processed_title'])
x = vect.transform(test['processed_title'])
rf_model = RandomForestClassifier()
rf_model.fit(X,train_df['topic_idx'])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [31]:
from sklearn.linear_model import RidgeClassifier
train_df = pd.read_csv(r'/content/drive/MyDrive/데이콘 뉴스분석/train_data_processed.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/test_data_processed.csv', index_col='Unnamed: 0')
vect = TfidfVectorizer(max_features =5000)
X = vect.fit_transform(train_df['processed_title'])
x = vect.transform(test['processed_title'])
ridge_model = RidgeClassifier()
ridge_model.fit(X,train_df['topic_idx'])


RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [35]:
vect = TfidfVectorizer()
X2 = vect.fit_transform(train_df['processed_title'])
x2 = vect.transform(test['processed_title'])

# 앙상블

### 딥러닝, 머신러닝 모델들의 결과를 정규화 한뒤 가중치를 줘서 더해서 결과를 냈습니다.

In [23]:
dnn_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/dnn_pred.npy')
dnn2_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/dnn_pred2.npy')
model2_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/model2_pred_title_15.npy')
model3_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/model3_pred_title_16.npy')
model3_2_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/model3(2)_pred_title_64_20.npy')
model3_3_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/model3_pred_processed_title_15.npy')
model4_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/kykim.npy')
model5_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/snunlp.npy')
model6_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/funnel.npy')
model7_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/funnel10.npy')
model8_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/kobert15_3e-5.npy')
model9_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/seongju.npy')
model10_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/seongjuprocessed1e.npy')
model11_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/snuprocessed6.npy')
model12_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/Electra6.npy')
model13_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/ElectraProcessed6.npy')
model14_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/robertalargenonetune.npy')
model15_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/electra15.npy')
model16_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/electra8.npy')
model17_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/electra864.npy')
model18_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/electraproccessed8.npy')
model19_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/funnel832.npy')
model20_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/funnel864.npy')
model21_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/kobert15_3e-5_datasplitchange.npy')
model22_pred = np.load('/content/drive/MyDrive/데이콘 뉴스분석/kobert15_3e-5_datasplitchange6.npy')

In [24]:
model2_result = (model2_pred - model2_pred.mean(axis = 1).reshape(9131,1))/model2_pred.std(axis= 1).reshape(9131,1)
dnn_result =  (dnn_pred - dnn_pred.mean(axis = 1).reshape(9131,1))/dnn_pred.std(axis= 1).reshape(9131,1)
dnn2_result =  (dnn2_pred - dnn2_pred.mean(axis = 1).reshape(9131,1))/dnn2_pred.std(axis= 1).reshape(9131,1)
model3_result = (model3_pred - model3_pred.mean(axis = 1).reshape(9131,1))/model3_pred.std(axis= 1).reshape(9131,1)
model3_2_result = (model3_2_pred - model3_2_pred.mean(axis = 1).reshape(9131,1))/model3_2_pred.std(axis= 1).reshape(9131,1)
model3_3_result = (model3_3_pred - model3_3_pred.mean(axis = 1).reshape(9131,1))/model3_3_pred.std(axis= 1).reshape(9131,1)
model4_result = (model4_pred - model4_pred.mean(axis = 1).reshape(9131,1))/model4_pred.std(axis= 1).reshape(9131,1)
model5_result = (model5_pred - model5_pred.mean(axis = 1).reshape(9131,1))/model5_pred.std(axis= 1).reshape(9131,1)
model6_result = (model6_pred - model6_pred.mean(axis = 1).reshape(9131,1))/model6_pred.std(axis= 1).reshape(9131,1)
model7_result = (model7_pred - model7_pred.mean(axis = 1).reshape(9131,1))/model7_pred.std(axis= 1).reshape(9131,1)
model8_result = (model8_pred - model8_pred.mean(axis = 1).reshape(9131,1))/model8_pred.std(axis= 1).reshape(9131,1)
model9_result = (model9_pred - model9_pred.mean(axis = 1).reshape(9131,1))/model9_pred.std(axis= 1).reshape(9131,1)
model10_result = (model10_pred - model10_pred.mean(axis = 1).reshape(9131,1))/model10_pred.std(axis= 1).reshape(9131,1)
model11_result = (model11_pred - model11_pred.mean(axis = 1).reshape(9131,1))/model11_pred.std(axis= 1).reshape(9131,1)
model12_result = (model12_pred - model12_pred.mean(axis = 1).reshape(9131,1))/model12_pred.std(axis= 1).reshape(9131,1)
model13_result = (model13_pred - model13_pred.mean(axis = 1).reshape(9131,1))/model13_pred.std(axis= 1).reshape(9131,1)
model14_result = (model14_pred - model14_pred.mean(axis = 1).reshape(9131,1))/model14_pred.std(axis= 1).reshape(9131,1)
model15_result = (model15_pred - model15_pred.mean(axis = 1).reshape(9131,1))/model15_pred.std(axis= 1).reshape(9131,1)
model16_result = (model16_pred - model16_pred.mean(axis = 1).reshape(9131,1))/model16_pred.std(axis= 1).reshape(9131,1)
model17_result = (model17_pred - model17_pred.mean(axis = 1).reshape(9131,1))/model17_pred.std(axis= 1).reshape(9131,1)
model18_result = (model18_pred - model18_pred.mean(axis = 1).reshape(9131,1))/model18_pred.std(axis= 1).reshape(9131,1)
model19_result = (model19_pred - model19_pred.mean(axis = 1).reshape(9131,1))/model19_pred.std(axis= 1).reshape(9131,1)
model20_result = (model20_pred - model20_pred.mean(axis = 1).reshape(9131,1))/model20_pred.std(axis= 1).reshape(9131,1)
model21_result = (model21_pred - model21_pred.mean(axis = 1).reshape(9131,1))/model21_pred.std(axis= 1).reshape(9131,1)
model22_result = (model22_pred - model22_pred.mean(axis = 1).reshape(9131,1))/model22_pred.std(axis= 1).reshape(9131,1)

### 최종 제출 모델

In [29]:
rf_proba = []
for i in list(rf_model.predict(x)):
  tmp_list = [0,0,0,0,0,0,0]
  tmp_list[i] = 1
  rf_proba.append(tmp_list)
np.array(rf_proba)

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [46]:
sub = pd.read_csv('/content/drive/MyDrive/데이콘 뉴스분석/sample_submission.csv', index_col='index')

sub['topic_idx'] =  np.argmax(1*model9_result +0.5 *model11_result + 0.5*dnn_result + 1 * model10_result + 0.2 * 
                                                       model8_result + 0.5 * model7_result + 2.7*model.predict_proba(x) +
                                                       1.2* model5.predict_proba(x) + 0.5* model3.predict_proba(x) + 
                                                       1.6*model8.predict_proba(x) + 
                                                       #2.3* model8_2.predict_proba(x2) +
                                                        0.3*np.array(rf_proba) +0.5 *model3_result 
                                                        + 0.9*model3_2_result+ 0.2*np.array(rf_proba) 
                                                        + 0.2*model5_2.predict_proba(x)
                                                        
                                                                                                          
                                                        ,axis = 1)

In [47]:
sub

,topic_idx
index,
45654,0
45655,3
45656,2
45657,2
45658,3
...,...
54780,3
54781,2
54782,2


In [48]:
sub.to_csv('앙상블_17.csv')